In [1]:
import numpy as np
import scipy as sp
from pyscf import gto, scf, ao2mo

# Background

Hartree-Fock Self-Consistent Field Theory (HF-SCF) with restricted orbitals and closed-shell systems (RHF) sovles the following pseduo eigenvalue problem:

$$FC = S C \epsilon$$

called the Roothan equations.

This is soolved self-consistently for orbital coefficient matrix $C$ and orbital energy eigenvalues $\epsilon$

The Fock matrix has elements $F_{\mu\nu}$ in the **atomic orbital basis** as :

$$F_{\mu\nu} = H_{\mu\nu} + 2 (\mu\nu|\lambda \sigma)D_{\lambda \sigma} -  (\mu\lambda|\nu \sigma)D_{\lambda \sigma}$$

where

$$D_{\lambda \sigma} = C_{\sigma i} C_{\lambda i}$$

Note $C$ is an $(N \times M)$ matrix, where $N$ is no. of **atomic basis** fucntions and $M$ the number of molecular orbitals

Note too that it is common to write Coulomb and Exchange matrices J and K, with elements (here square brackets denote their dependents on the density matrix!):

$$J[D_{\lambda \sigma}]_{\mu\nu} = (\mu\nu|\lambda \sigma)D_{\lambda \sigma}$$

$$K[D_{\lambda \sigma}]_{\mu\nu} = (\mu\lambda|\nu \sigma)D_{\lambda \sigma}$$

The Fock matrix can therefore be found by:

$$F = H + 2J -  K$$

# Manual Calculation!

### 1. Define molecule in PySCF

In [2]:
# geometry = """
# O
# H 1 1.1
# H 1 1.1 2 104
# """
# # basis =  'cc-pvdz'
# basis =  'STO-3G'

In [3]:
geometry = """
H 0 0 0 
H 0 0 0.74
"""
basis =  'STO-3G'

In [4]:


full_system_mol = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol.build()

In [5]:
full_system_scf = scf.RHF(full_system_mol)
full_system_scf.verbose=1
full_system_scf.max_memory= 8_000
full_system_scf.conv_tol = 1e-6

E_HF_Pyscf = full_system_scf.kernel()
E_HF_Pyscf

-1.1167593073964255

### 2. Obtain MO coefficients and MO energies

Remember that HF theroy give use optimized **spatial** molecular orbitals: $\{ \psi_{i} \}$

where:

$$\psi_{i} = \sum_{\mu=1}^{K} C_{\mu i} \phi_{mu}$$

In [6]:
C = full_system_scf.mo_coeff 

C # columns are MOs!

array([[ 0.54884228,  1.21245192],
       [ 0.54884228, -1.21245192]])

Next we want to convert our integrals over (spatial) AOs into MO (spatial) basis!

$$(\psi_{i} \psi_{j}|\psi_{k} \psi_{l}) = \sum_{\mu} \sum_{\nu} \sum_{\lambda} \sum_{\sigma} C^{*}_{\mu i} C_{\nu j} C^{*}_{\lambda k} C_{\sigma l} (\mu \nu | \lambda \sigma )$$

In [7]:
eri_ao = full_system_mol.intor('int2e') # 2e- electron repulsion integrals in AO basis <ij|kl>

In [8]:
tmp = np.einsum('pi,pqrs->iqrs', C, eri_ao, optimize=True)
tmp = np.einsum('qa,iqrs->iars', C, tmp, optimize=True)
tmp = np.einsum('iars,rj->iajs', tmp, C, optimize=True)
ERI_mo_basis = np.einsum('iajs,sb->iajb', tmp, C, optimize=True)
del tmp
ERI_mo_basis.shape

(2, 2, 2, 2)

In [9]:
## PYSCF CHECK

pyscf_mo_ints = ao2mo.kernel(full_system_mol, full_system_scf.mo_coeff)#, aosym=1)

# Convert the 2e integrals (in Chemist’s notation) 
pyscf_mo_ints = ao2mo.restore(1, pyscf_mo_ints, full_system_mol.nao)


# pyscf_mo_ints = pyscf_mo_ints.transpose(0,2,3,1) #<- converts to physists notation if needed
pyscf_mo_ints.shape
np.allclose(ERI_mo_basis, pyscf_mo_ints)

True

Currently we have converted AO-basis integrals into spatial-MO basis integrals!

BUT we would also like this in the spin-orbital basis!

In [10]:
# IMPORTANT we can order spins in different ways!

# here we order as spin_up, spin_down alternating (but could do all spin up followed by all spin down)!

In [11]:
n_spatial_orbs = full_system_scf.mol.nao
n_spin_orbs = 2* n_spatial_orbs
ERI_mo_basis_spin = np.zeros((n_spin_orbs,n_spin_orbs,n_spin_orbs,n_spin_orbs))

##
for p in range(n_spatial_orbs):
    for q in range(n_spatial_orbs):
        for r in range(n_spatial_orbs):
            for s in range(n_spatial_orbs):
               
                AO_term = ERI_mo_basis[p,q,r,s] 
                # up,up,up,up 
                ERI_mo_basis_spin[2*p,2*q,2*r,2*s] = AO_term
                # down,down, down, down
                ERI_mo_basis_spin[2*p+1,2*q+1,2*r+1,2*s+1] = AO_term
                
                #  up up down down
                ERI_mo_basis_spin[2*p,2*q,2*r+1,2*s+1] =  AO_term
                # down down up up 
                ERI_mo_basis_spin[2*p+1,2*q+1,2*r,2*s] =  AO_term
                
                # other mixed terms go to zero!
                # see Szabo eq 2.165 pg 82

ERI_mo_basis_spin.nonzero()[0].shape

(64,)

In [12]:
# need to do the same for one body terms
H_core_AO = full_system_scf.get_hcore()
h_core_spatial_basis =  C.conj().T @ H_core_AO @ C # <psi H psi> # NOT FOCK
h_core_spatial_basis.shape

(2, 2)

In [13]:
# convert to MO spin basis
h_core_mo_basis_spin = np.zeros((n_spin_orbs, n_spin_orbs))
for p in range(n_spatial_orbs):
    for q in range(n_spatial_orbs):
        
        # populate 1-body terms (must have same spin, otherwise orthogonal)
        ## pg 82 Szabo
        h_core_mo_basis_spin[2*p, 2*q] = h_core_spatial_basis[p,q] # spin UP
        h_core_mo_basis_spin[(2*p + 1), (2*q +1)] = h_core_spatial_basis[p,q] # spin DOWN
        
np.around(h_core_mo_basis_spin,3)

array([[-1.253,  0.   ,  0.   ,  0.   ],
       [ 0.   , -1.253,  0.   ,  0.   ],
       [-0.   ,  0.   , -0.475,  0.   ],
       [ 0.   , -0.   ,  0.   , -0.475]])

In [14]:
np.around(C.conj().T @ full_system_scf.get_hcore() @ C , 3) # should be doubled version of this!

array([[-1.253,  0.   ],
       [-0.   , -0.475]])

test --> Lets build fock spin matrix

(szabo pg 68 for ket notation!) note chemist vs physist rules!

$$f_{pq} = h_{pq} + \sum_{m}^{occ} \langle pm || qm \rangle = h_{pq} +\sum_{m}^{occ} \big( \langle pm | qm \rangle  - \langle pm| \underbrace{mq}_{order} \rangle  \big) = h_{pq} +\sum_{m}^{occ} \big( [ pq | mm ]  - [ pm| mq ]  \big) $$  

In [21]:
## test. Lets build fock spin matrix

# Fock_spin_mo = np.zeros((n_spin_orbs, n_spin_orbs))

n_electrons = full_system_scf.mol.nelectron

# note this is in chemist notation (look at slices!)

pm_qm = np.einsum('pqmm->pq', ERI_mo_basis_spin[:            ,   :,
                                                :n_electrons ,   :n_electrons])
pm_mq = np.einsum('pmmq->pq', ERI_mo_basis_spin[:            ,   :n_electrons,
                                                :n_electrons ,    :])

Fock_spin_mo = h_core_mo_basis_spin + (pm_qm - pm_mq)

np.around(Fock_spin_mo, 3)

array([[-0.579,  0.   , -0.   ,  0.   ],
       [ 0.   , -0.579,  0.   , -0.   ],
       [ 0.   ,  0.   ,  0.671,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.671]])

In [22]:
np.around(C.conj().T @ full_system_scf.get_fock() @ C , 3) # again should be doubled version of this!

array([[-0.579,  0.   ],
       [ 0.   ,  0.671]])

In [23]:
full_system_scf.mo_energy

array([-0.57855386,  0.67114349])

In [18]:
# diagonal enteries of Fock_spin_mo are energies of orbitals!

In [19]:
asdf

NameError: name 'asdf' is not defined

# MP2

In [69]:
from pyscf import mp

pyscf_mp2 = mp.MP2(full_system_scf)
pyscf_mp2.verbose = 0
pyscf_mp2.run()
pyscf_mp2.e_tot

-1.1298973809859585

In [99]:
pyscf_mp2.e_tot - pyscf_mp2.e_hf #  + full_system_scf.energy_nuc()

-0.013138073589533006

In [100]:
pyscf_mp2.emp2

-0.013138073589532983

In [70]:
from pyscf.cc.addons import spatial2spin

T2_amps = spatial2spin(pyscf_mp2.t2)
T2_amps

NPArrayWithTag([[[[ 0.        ,  0.        ],
                  [ 0.        ,  0.        ]],

                 [[ 0.        , -0.07250174],
                  [ 0.07250174,  0.        ]]],


                [[[ 0.        ,  0.07250174],
                  [-0.07250174,  0.        ]],

                 [[ 0.        ,  0.        ],
                  [ 0.        ,  0.        ]]]])

# MP2  Energy and Amps

$$E_{MP2} = \frac{1}{4} \sum_{ij}^{occ} \sum_{ab}^{virt} t_{ijab}\: \langle ij || ab \rangle  $$


$$t_{ijab} = \frac{\langle ij || ab \rangle}{e_{i}+e_{i}-e_{a}-e_{b}} = \frac{ \langle ij | ab \rangle  - \langle ij| ba \rangle   }{e_{i}+e_{i}-e_{a}-e_{b}}$$

and in chemist notation:


$$t_{ijab} = \frac{  [ ia | jb ]  -[ ib| ja ]) }{e_{i}+e_{i}-e_{a}-e_{b}}$$



However for **CLOSED shell calculations the formula is different**:


$$E_{MP2} =  \frac{1}{4} \sum_{ij}^{occ} \sum_{ab}^{virt} \frac{ \langle ia | jb \rangle \Big( 2\langle ia | jb \rangle - \langle ib | ja \rangle \Big) }{e_{i}+e_{i}-e_{a}-e_{b}}$$

$$E_{MP2} =  \frac{1}{4} \sum_{ij}^{occ} \sum_{ab}^{virt} \frac{ [ij | ab ] \Big( 2 [ ij | ab ] - [ ij | ba ] \Big) }{e_{i}+e_{i}-e_{a}-e_{b}} $$

In [131]:
e_orbs_occ =  np.diag(Fock_spin_mo)[:n_electrons]

e_i = e_orbs_occ.reshape(-1, 1, 1, 1)
e_j = e_orbs_occ.reshape(1, -1, 1, 1)

e_orbs_vir = np.diag(Fock_spin_mo)[n_electrons:]
e_a = e_orbs_vir.reshape(1, 1, -1, 1)
e_b = e_orbs_vir.reshape(1, 1, 1, -1)

# abgij = g[nocc:, nocc:, :nocc, :nocc]

In [133]:
# note how ij_ab == ij_ba in CLOSED shell calcs
ij_ab = ERI_mo_basis_spin[:n_electrons, :n_electrons, 
                          n_electrons:, n_electrons:]


ij_ab_squared = ij_ab@ij_ab

fraction = ij_ab_squared * (1.0 / (e_i + e_j - e_a - e_b) )

E_mp2 = 0.25 * np.einsum('ijab->', fraction)
E_mp2

-0.17624780258503414

In [134]:
(ibja) * 1.0 / (e_i + e_j - e_a - e_b)

array([[[[-0.07250174, -0.        ],
         [-0.        , -0.07250174]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]]],


       [[[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.07250174, -0.        ],
         [-0.        , -0.07250174]]]])

In [52]:
iajb = ERI_mo_basis_spin[:n_electrons, n_electrons:, 
                         :n_electrons, n_electrons:]

ibja = ERI_mo_basis_spin[:n_electrons, n_electrons:, 
                         :n_electrons, n_electrons:]

iajb-ibja

array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]])

In [24]:
aibj = ERI_mo_basis_spin[n_electrons:, :n_electrons, 
                         n_electrons:, :n_electrons]

ajbi = ERI_mo_basis_spin[n_electrons:, :n_electrons, 
                         n_electrons:, :n_electrons]

aibj-ajbi

array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]])

In [33]:
orb_energies = np.diag(Fock_spin_mo)
e_orbs_occ =  orb_energies[:n_electrons]

e_i = e_orbs_occ.reshape(1, 1, -1, 1)
e_j = e_orbs_occ.reshape(1, 1, 1, -1)

e_orbs_vir = orb_energies[n_electrons:]
e_a = e_orbs_vir.reshape(-1, 1, 1, 1)
e_b = e_orbs_vir.reshape(1, -1, 1, 1)

In [35]:
a_abji =ERI_mo_basis_spin[n_electrons:, n_electrons:, 
                          :n_electrons, :n_electrons]
amplitudes = a_abji * 1.0 / (e_i + e_j - e_a - e_b)

amplitudes

array([[[[-0.26554885, -0.        ],
         [-0.        , -0.26554885]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]]],


       [[[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.26554885, -0.        ],
         [-0.        , -0.26554885]]]])

In [38]:
E = np.einsum('abij,abij->', amplitudes, a_abji) - np.einsum('abji,abij', amplitudes, a_abji,
                                                                        optimize='greedy')
E

0.0

In [28]:
# one_body_integrals = H_ferm._one_body_integrals
# two_body_integrals = H_ferm._two_body_integrals

# g = two_body_integrals

e_orbs_occ =  np.diag(Fock_spin_mo)[:n_electrons]

e_i = e_orbs_occ.reshape(1, 1, -1, 1)
e_j = e_orbs_occ.reshape(1, 1, 1, -1)

e_orbs_vir = np.diag(Fock_spin_mo)[n_electrons:]
e_a = e_orbs_vir.reshape(-1, 1, 1, 1)
e_b = e_orbs_vir.reshape(1, -1, 1, 1)

# abgij = g[nocc:, nocc:, :nocc, :nocc]


In [31]:
e_orbs_vir = fij[nocc:]
e_a = e_orbs_vir.reshape(-1, 1, 1, 1)
e_b = e_orbs_vir.reshape(1, -1, 1, 1)

abgij = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (e_i + e_j - e_a - e_b)
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

array([-0.57855386, -0.57855386])

In [ ]:
asdfasd

In [68]:
phys_ints = ERI_mo_basis_spin.transpose(0,2,3,1) #<- converts to physists notation if needed

ij_ab = phys_ints[:n_electrons, :n_electrons, n_electrons:, n_electrons:]
ij_ba = phys_ints[:n_electrons, :n_electrons, n_electrons:, n_electrons:]

ij_ab - ij_ba

array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]])

In [ ]:
n_qubits = 2*hcore_ij.shape[0]

one_body_terms = np.zeros((n_qubits, n_qubits))
two_body_terms = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))

for p in range(n_qubits//2):
    for q in range(n_qubits//2):
        
        # populate 1-body terms (must have same spin, otherwise orthogonal)
        ## pg 82 Szabo
        one_body_terms[2*p, 2*q] = hcore_ij[p,q] # spin UP
        one_body_terms[(2*p + 1), (2*q +1)] = hcore_ij[p,q] # spin DOWN
        
        # continue 2-body terms
        for r in range(n_qubits//2):
            for s in range(n_qubits//2):
                                
                ### SAME spin                
                two_body_terms[2*p, 2*q , 2*r, 2*s] = ERI_mo_basis[p,q,r,s] # up up up up
                two_body_terms[(2*p+1), (2*q +1) , (2*r + 1), (2*s +1)] = ERI_mo_basis[p,q,r,s] # down down down down
                
                ### mixed spin                
                two_body_terms[2*p, 2*q , (2*r + 1), (2*s +1)] = ERI_mo_basis[p,q,r,s] # up up down down
                two_body_terms[(2*p+1), (2*q +1) , 2*r, 2*s] = ERI_mo_basis[p,q,r,s] # down down up up 
               
                # other mixed terms go to zero!
                # see Szabo eq 2.165 pg 82
                
                
                
### remove vanishing terms
EQ_Tolerance=1e-8
one_body_terms[np.abs(one_body_terms)<EQ_Tolerance]=0
two_body_terms[np.abs(two_body_terms)<EQ_Tolerance]=0

In [ ]:
S = full_system_mol.intor('int1e_ovlp') # 1e- electron overlap <i|j>

eri_ao = full_system_mol.intor('int2e') # 2e- electron repulsion integrals in AO basis <ij|kl>

In [ ]:
n_docc = full_system_mol.nelectron // 2 # number of double occupied orbitals
n_bas_ft = full_system_mol.nao

print(f'Number of occupied orbitals:{n_docc:4.0f}')
print(f'Number of basis functions: {n_bas_ft:6.0f}')

### 3. Build Core Hamiltonian

In [ ]:
T = full_system_mol.intor('int1e_kin')
V = full_system_mol.intor('int1e_nuc')

H_core = T+V

In [ ]:
# check H_core calculated is the SAME!
np.allclose(H_core, scf.hf.get_hcore(full_system_mol))

### 4. Check roothan and S matrix

We can solve:

$$FC = S C \epsilon$$

if $S$ is the identity matrix... however:

In [ ]:
test = np.allclose(S, np.eye(S.shape[0]))
print(f'is S the identity matrix?: {test}')
if test is False:
    print()
    print('AO basis is not orthonormal')

Overall we cannot ignore the AO overlap matrix... $F$ cannot simply be diagonlized to solve for the orbital coefficent matrix!

LUCKILY: we can overcome this issue, by **transforming** the AO basis so that all the basis functions are orthonormal!

aka we need a transform:

$$U^{\dagger} S U = \mathcal{I}$$

Clearly this makes S diagonal!

But how do we do this? (pg 143-144 Szabo and https://www.youtube.com/watch?v=2N104Nf-_L4)

1. Symmetric orthogonalization
    - uses the symmetric inverse square root of the overlap matrix
    - $ U = S^{-\frac{1}{2}}$

as clearly:

$$U^{\dagger} S U = S^{-\frac{1}{2}} S S^{-\frac{1}{2}}  = S^{-\frac{1}{2}} S^{+\frac{1}{2}} = \mathcal{I}$$

**IMPORTANT numerical problems can occur if the overlap matrix has small eigenvalues, which may occur for large systems or for systems where diffuse basis sets are used**

2. canonical orthogonalization
    - uses the symmetric inverse square root of the overlap matrix and additional unitary $V$
    - V is the unitary we use to diagonlize the matrix $S$
        - we then truncate according to the eignvalues of $V$... for small terms there will be numerical problems so they are REMOVED (TRUNCATED)
    - $ U = V S^{-\frac{1}{2}}$
    
This numerical problem may be avoided by using canonical orthogonalization, in which an asymmetric inverse square root of the overlap matrix is formed, with numerical stability enhanced by the elimination of eigenvectors corresponding to very small eigenvalues. As a few combinations of AO basis functions may be discarded, the number of canonical-orthogonalized OSOs and MOs may be slightly smaller than the number of AOs.



3. Cholesky decomposition

When the basis set is too overcomplete, the eigendecomposition of the overlap matrix is no longer numerically stable. In this case the partial Cholesky decomposition can be used to pick a subset of basis functions that span a sufficiently complete set, see [Lehtola:2019:241102] and [Lehtola:2020:032504]. This subset can then be orthonormalized as usual; the rest of the basis functions are hidden from the calculation. The Cholesky approach allows reaching accurate energies even in the presence of significant linear dependencies [Lehtola:2020:04224].



### 5. Let us use Symmetrix orthogonalization

In [ ]:
U = sp.linalg.fractional_matrix_power(S, -0.5)

### check orthonormal condition

S_prime = U @ S @ U
print(f'is S_prime orthonormal: {np.allclose(S_prime, np.eye(S_prime.shape[0]))}')

This is good, as we can now think about diagonalization!


The drawback of this scheme is that we would now have to either re-compute the ERI and core Hamiltonian tensors in the newly orthogonal AO basis, or transform them using our $U$ matrix (both would be overly costly, especially transforming the ERI)

However we can directly subsitute $C = U C_{prime}$ into our Roothan equations!:

$$FC = S C \epsilon$$
$$F U C_{prime} = S U C_{prime} \epsilon$$

now apply $U^{\dagger}$ on both sides

$$(U^{\dagger}F U) C_{prime} = (U^{\dagger} S U) C_{prime} \epsilon$$
$$F_{prime} C_{prime} = \mathcal{I} C_{prime} \epsilon$$

$$F_{prime} C_{prime} = C_{prime} \epsilon$$


We have a standard eigenvalue equation now!!!

We can solve for the transformed orbtial coefficient matrix $C_{prime}$ by diagonalizing the transformed Fock matrix $F_{prime}$...

Then simply transform  $C_{prime}$ back to AO basis using  $C = U C_{prime}$  ( as started in the beginning)

### 6. But how do we start?

In order to find the Fock matrix we need the orbital coefficient matrix $C$, but in order to compute $C$ we need $F$...

$$F_{\mu\nu} = H_{\mu\nu} + 2 (\mu\nu|\lambda \sigma)C_{\sigma i} C_{\lambda i} -  (\mu\lambda|\nu \sigma)C_{\sigma i} C_{\lambda i}$$

We don't know $F$ or $C$... to begin we therefore GUESS the Fock matrix to obtain an initial $C$ matrix

Often a logical starting point to guessing the Fock matrix, is to start from the only part that doesn't depend on the $C$ matrix (as seen in the eq. above this is the Core Hamitlonian part!)

1. start with $F = H_{core}$ (APPROXIMATION)
2. Diagonalize transformed Fock matrix $U^{\dagger}F U)$
3. This gives initial C_prime
4. transform back to AO basis

In [ ]:
F_guess = H_core

F_prime = U.conj().T @ H_core @ U

epsilon, C_prime = np.linalg.eigh(F_prime)

# transofrm C_prime back to AO basis
C = U @ C_prime

# get occupied orbitals
C_occ = C[:, :n_docc]

# build density matrix from occupied orbitals!
D = C_occ @ C_occ.T
# D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)

### 7. Perform SCF routine

1. Build Fock matrix
    - build coulomb matrix $J$
    - build exchange matrix $K$  
    - $F = H_{core} + 2J - K$
2. Find RHF energy (Szabo pg 150 (pg 134 has similar eq BUT in MO basis!)

$$E_{0} = \frac{1}{2} \sum_{\mu} \sum_{\nu} D_{\mu \nu} (H_{core} + F_{\mu \nu})$$

    - check convergence
    - if true: Break
    - else: go to 3.
3. Compute new orbital guess
    - Transform F to orthonormal AO basis
    - Diagonalize F_prime to give epsilon and C_prime
    - transform C_prime back to AO basis
    - Get new Density matrix from occupied orbitals of C

In [ ]:
#alg

max_iter = 100

HF_energy =0
E_previous = 0
E_tol = 1e-6

for i in range(max_iter+1):
    
    # Build Fock Matrix
    J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
    K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D, optimize=True)
    
    Fock = H_core + 2*J_mat - K_mat
    
    ### find RHF energy
    HF_energy = np.einsum('pq,pq ->', D, (Fock+H_core), optimize=True) + full_system_mol.energy_nuc()
#     HF_energy =np.trace(H_core @ D) + np.trace(Fock @ D) + full_system_mol.energy_nuc()
    # (no 0.5 here as included in D mat natively!)
    
    ### check convergence
    if np.abs(HF_energy-E_previous)<E_tol:
        break
        
        
    # if not convereged store old result
    E_previous = HF_energy
    
    ## compute new orbital guess
    F_prime = U.conj().T @ Fock @ U

    epsilon, C_prime = np.linalg.eigh(F_prime)

    # transofrm C_prime back to AO basis
    C = U @ C_prime

    # get occupied orbitals
    C_occ = C[:, :n_docc]

    # build density matrix from occupied orbitals!
#     D = C_occ @ C_occ.T
    D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
    
    if i==max_iter:
        raise ValueError('Maximum number of SCF iterations exceeded')
        
print(f'final RHF SCF energy {HF_energy}')

In [ ]:
##### check FC = SCe

# Build Fock Matrix
J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D, optimize=True)

Fock = H_core + 2*J_mat - K_mat

np.allclose(Fock@C[:,0],epsilon[0]*(S@C)[:,0])

# Compare to PySCF result!

In [ ]:
full_system_scf = scf.RHF(full_system_mol)
full_system_scf.verbose=1
full_system_scf.max_memory= 8_000
full_system_scf.conv_tol = 1e-6

E_HF_Pyscf = full_system_scf.kernel()

In [ ]:
print(f'difference in Results: {np.abs(E_HF_Pyscf - HF_energy) : 0.9f}')

In [ ]:
# Note certain properties!

In [ ]:
# By definition cannonical HF MOs are eigenfucntions of the Fock operator

# Build Fock Matrix in AO basis
J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D, optimize=True)
Fock = H_core + 2*J_mat - K_mat

# convert to MO basis
F_MO =  C.conj().T @ Fock @ C
# F_MO_2 = np.einsum('mi,vj, mv->ij', C.conj(), C, Fock, optimize=True)
# print(np.allclose(F_MO, F_MO_2))



np.around(F_MO, 4)  # should be diagonal, with orbital energies diagonals! see Szabo pg 165

In [ ]:
epsilon

In [ ]:
# check diagonals are orbital energies!
np.allclose(np.diag(F_MO), epsilon)

# IMPORTANT notes

PG 134 and 164 Szabo

Remember that HF theroy give use optimized molecular orbitals: $\{ \psi_{i} \}$

where:

$$\psi_{i} = \sum_{\mu=1}^{K} C_{\mu i} \phi_{mu}$$

- note the set $\{ \phi_{i}  | i = 1, 2 ... K\}$ are the known basis functions!
- H Fock gives us optimal $C$ matrix
    - where columns of $C$ from HF calculation give the molecular orbitals!

Note in Hfock calculation we use AO orbitals to find the energy (Szabo pg 141):

$$F_{\mu\nu} = H_{\mu\nu} + 2 (\mu\nu|\lambda \sigma)C_{\sigma i} C_{\lambda i} -  (\mu\lambda|\nu \sigma)C_{\sigma i} C_{\lambda i}$$

- where :
    - $(\mu\lambda|\nu \sigma) = \int d\vec{r}_{1} d\vec{r}_{2} \phi^{*}_{\mu}(1) \phi_{\nu}(1) \frac{1}{r_{12}}  \phi^{*}_{\lambda}(2) \phi_{\sigma}(2)$
    - aka both H_core and two electron integrals are found by **integrals over AO basis functions** ($\{ \phi_{i}  | i = 1, 2 ... K\}$)!
    
However if we want to do post HF methods, we often write the solution to our problem using Slater determinants - that contain molecular orbitals (NOT atomic orbs)!

$$| \Psi^{HF}> = | \psi_{1} \bar{\psi_{1}} \: \: \psi_{2} \bar{\psi_{2}} \: \: ... \: \: \psi_{N/2} \bar{\psi_{N/2}}>$$

Therefore we need to convert to the MO basis!

Note $\psi_{i}$ is a molecular orbital! (not atomic!)

SO the 1e- integrals become:

$$h_{ij} = (\psi_{i} |h| \psi_{j}) = \sum_{\mu} \sum_{\nu} C^{*}_{\mu i} C_{\nu j} H_{\mu \nu}^{\text{core AO basis}}$$

Szabo pg 150!


$$<O_{1}> = <\Psi_{i} |O_{1} |\Psi_{i}> = \sum_{a}^{N/2}  (\psi_{a} |h| \psi_{a}) =  \sum_{\mu}^{N/2} \sum_{\nu}^{N/2} C^{*}_{\mu i} C_{\nu j} H_{\mu \nu}^{\text{core AO basis}}$$

In [ ]:
# Szabo pg 150!

hcore_ij =  C.conj().T @ H_core @ C # <psi H psi> # NOT FOCK
# hcore_ij = np.einsum('mi,vj, mv->ij', C.conj(), C, H_core, optimize=True)

AA = np.einsum('ii->', hcore_ij[:n_docc, :n_docc], optimize=True)
BB = np.einsum('pq,pq ->', D, H_core, optimize=True)

np.isclose(AA, BB)

notice how MO basis is much easier to evaluate $<O_{1}>$... just use Slater rules (which turns out to be diagonal elements of hij)! 

Note $\psi_{i}$ is a molecular orbital! (not atomic!)

SO the 2e- integrals become:

$$(\psi_{i} \psi_{j}|\psi_{k} \psi_{l}) = \sum_{\mu} \sum_{\nu} \sum_{\lambda} \sum_{\sigma} C^{*}_{\mu i} C_{\nu j} C^{*}_{\lambda k} C_{\sigma l} (\mu \nu | \lambda \sigma )$$

In [ ]:
# transform 2e- ints to MO basis!

# Cocc = C[:, :n_docc] # occupied
# Cvirt = C[:, n_docc:] # unoccupied

# tmp = np.einsum('pi,pqrs->iqrs', Cocc, eri_ao, optimize=True)
# tmp = np.einsum('qa,iqrs->iars', Cvirt, tmp, optimize=True)
# tmp = np.einsum('iars,rj->iajs', tmp, Cocc, optimize=True)
# ERI_mo_basis = np.einsum('iajs,sb->iajb', tmp, Cvirt, optimize=True)
# mo_ints.shape

tmp = np.einsum('pi,pqrs->iqrs', C, eri_ao, optimize=True)
tmp = np.einsum('qa,iqrs->iars', C, tmp, optimize=True)
tmp = np.einsum('iars,rj->iajs', tmp, C, optimize=True)
ERI_mo_basis = np.einsum('iajs,sb->iajb', tmp, C, optimize=True)
del tmp
ERI_mo_basis.shape

In [ ]:
pyscf_mo_ints = ao2mo.kernel(full_system_mol, full_system_scf.mo_coeff)#, aosym=1)

# Convert the 2e integrals (in Chemist’s notation) 
pyscf_mo_ints = ao2mo.restore(1, pyscf_mo_ints, n_bas_ft)


# pyscf_mo_ints = pyscf_mo_ints.transpose(0,2,3,1) #<- converts to physists notation if needed
pyscf_mo_ints.shape

This is 4D array for electron repulsion integrals in the MO representations...


Such MO integrals are required for all electron correlation methods. The two-electron AO integrals are the most numerous and the above equation appears to involve a computational effect proportional to M^8 (M^4 AO integrals each multiplied by four sets of M basis MO coefficients). However, by performing the transformation one index at a time, the computational effort can be reduced to M^5.

Each  step  now  only  involves  multiplication  of  M^4 basis integrals  with  M basis coefficients, i.e. the M^8 basis dependence is reduced to four M^5 operations. In the large basis set limit, all electron correlation methods formally scale as at least M 5basis, since this is the scaling for the AO to MO integral transformation. The transformation is an example of a “rotation” of the “coordinate” system consisting of the AOs, to one where the Fock operator is diagonal, the MOs (see Section 16.2). The diagonal system allows a much more compact representation  of  the  matrix  elements  needed  for  the  electron  correlation treatment. The coordinate change is also known as a four index transformation, since it involves four indices associated with the basis functions.

Szabo pg 84:

$$E_{0}^{\text{HF}} = 2 \sum_{a}^{N/2} (\psi_{a} |h^{core} | \psi_{a}) + \sum_{a}^{N/2} \sum_{b}^{N/2} \Big( 2(\psi_{a} \psi_{a}|\psi_{b} \psi_{b}) - (\psi_{a} \psi_{b}|\psi_{b} \psi_{a})\Big)$$


Szabo pg 85:

- $J_{ab} = (\psi_{a} \psi_{a}|\psi_{b} \psi_{b}) = <\psi_{a} \psi_{b}|\psi_{a} \psi_{b}> $
- $K_{ab} = (\psi_{a} \psi_{b}|\psi_{b} \psi_{a}) = <\psi_{a} \psi_{b}|\psi_{b} \psi_{a}> $

$$E_{0}^{\text{HF}} = 2 \sum_{a}^{N/2} h^{core}_{aa} + \sum_{a}^{N/2} \sum_{b}^{N/2} \Big( 2J_{ab} - K_{ab} \Big)$$

In [ ]:
## szabo pg 73! (eq 2.111) [may need eq 2.176 pg 84!]

# hcore_ij =  C.conj().T @ H_core @ C # <psi H psi> # NOT FOCK
# hcore_ij = np.einsum('mi,vj, mv->ij', C.conj(), C, H_core, optimize=True)

E = 2*np.einsum('ii->', hcore_ij[:n_docc, :n_docc], optimize=True)

two_J = 2*np.einsum('iijj->', pyscf_mo_ints[:n_docc, :n_docc, :n_docc, :n_docc], optimize=True)
K = np.einsum('ijji->', pyscf_mo_ints[:n_docc, :n_docc, :n_docc, :n_docc], optimize=True)
E+= (two_J-K)

E+= full_system_mol.energy_nuc()

E

In [ ]:
np.abs(E_HF_Pyscf - E)

In [ ]:
## szabo pg 73! (eq 2.111) [may need eq 2.176 pg 84!]

hcore_ij =  C.conj().T @ H_core @ C # <psi H psi> # NOT FOCK

E_manual_loop = 0
for i in range(n_docc):
    E_manual_loop+= 2*hcore_ij[i,i]

for i in range(n_docc):
    for j in range(n_docc):
        E_manual_loop+= (2*pyscf_mo_ints[i,i,j,j] - pyscf_mo_ints[i,j,j,i])

E_manual_loop+= full_system_mol.energy_nuc()
E_manual_loop

In [ ]:
np.abs(E_HF_Pyscf - E_manual_loop)

# NEXT convert the spatial MOs to spin MOs 

Previously we have spatial MO orbitals in our calculation:

$$| \Psi^{HF}> = | \psi_{1} \bar{\psi_{1}}, \: \: \psi_{2} \bar{\psi_{2}}, \: \: ... \: \:, \psi_{N/2} \bar{\psi_{N/2}}>$$

we can multiply our spatial orbitals by spin function (Szabo pg 47):

$$  \left.\begin{aligned}
  \chi_{2i}(\vec{x})&= \psi_{i}^{\alpha}(\vec{r}) \alpha(\omega)\\
  \chi_{2i-1}(\vec{x})&= \psi_{i}^{\beta}(\vec{r}) \beta(\omega)
\end{aligned}\right\} i = 1,2,..., k$$


Therefore our wavefunction in spin orbitals becomes (Szabo pg 83):

$$| \Psi^{HF}> = | \chi_{1} \: \: \chi_{2}  \: \: ... \: \:  \chi_{N-1} \: \: \chi_{N} >$$

In [ ]:
n_qubits = 2*hcore_ij.shape[0]

one_body_terms = np.zeros((n_qubits, n_qubits))
two_body_terms = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))

for p in range(n_qubits//2):
    for q in range(n_qubits//2):
        
        # populate 1-body terms (must have same spin, otherwise orthogonal)
        ## pg 82 Szabo
        one_body_terms[2*p, 2*q] = hcore_ij[p,q] # spin UP
        one_body_terms[(2*p + 1), (2*q +1)] = hcore_ij[p,q] # spin DOWN
        
        # continue 2-body terms
        for r in range(n_qubits//2):
            for s in range(n_qubits//2):
                                
                ### SAME spin                
                two_body_terms[2*p, 2*q , 2*r, 2*s] = ERI_mo_basis[p,q,r,s] # up up up up
                two_body_terms[(2*p+1), (2*q +1) , (2*r + 1), (2*s +1)] = ERI_mo_basis[p,q,r,s] # down down down down
                
                ### mixed spin                
                two_body_terms[2*p, 2*q , (2*r + 1), (2*s +1)] = ERI_mo_basis[p,q,r,s] # up up down down
                two_body_terms[(2*p+1), (2*q +1) , 2*r, 2*s] = ERI_mo_basis[p,q,r,s] # down down up up 
               
                # other mixed terms go to zero!
                # see Szabo eq 2.165 pg 82
                
                
                
### remove vanishing terms
EQ_Tolerance=1e-8
one_body_terms[np.abs(one_body_terms)<EQ_Tolerance]=0
two_body_terms[np.abs(two_body_terms)<EQ_Tolerance]=0

Szabo pg 83 eq 2.169 (energy in SPIN MOs):

$$E_{0}^{\text{HF}} = \sum_{a}^{N} (\chi_{a} |h^{\text{one body}} | \chi_{a}) + \frac{1}{2} \sum_{a}^{N} \sum_{b}^{N} \Big( (\chi_{a} \chi_{a}|\chi_{b} \chi_{b}) - (\chi_{a} \chi_{b}|\chi_{b} \chi_{a})\Big)$$

In [ ]:
np.around(one_body_terms, 4) # spins seperate (up, down, up, down, up, down)

In [ ]:
N = full_system_mol.nelectron

E_spin_MOs = np.einsum('ii->', one_body_terms[:N, :N], optimize=True)

J = np.einsum('iijj->', two_body_terms[:N, :N, :N, :N], optimize=True)
K = np.einsum('ijji->', two_body_terms[:N, :N, :N, :N], optimize=True)
E_spin_MOs+= 0.5*(J-K)

E_spin_MOs+= full_system_mol.energy_nuc()

E_spin_MOs

In [ ]:
np.abs(E_HF_Pyscf - E_spin_MOs)

Note in above we are taking sums with a limit of $N$, this is like considering:

$$| \Psi^{HF}> = | \chi_{1} \: \: \chi_{2}  \: \: ... \: \:  \chi_{N-1} \: \: \chi_{N} > = | 1^{(1)} 1^{(2)} ... 1^{(N)} 0^{(N+1)} 0^{(N+2)} ... 0^{(2K)}> $$

(only first N sites filled = HF ground state)

We can take sums over different multi-configurational ground states to give electron correlation energies! (Just change indices of summations). Note normalizations will be important!

# Qubit Hamiltonian

(Szabo pg 95... important to note physists and chemist notation differences!)

$$H_{q} = \text{constant}_{\text{nuclear}} + \sum_{p} \sum_{q} h_{pq}^{\text{one body}} a^{\dagger}_{p}a_{q} + \frac{1}{2} \sum_{p} \sum_{q} \sum_{r} \sum_{s} g_{pqrs}^{\text{two body}}  a^{\dagger}_{p} a^{\dagger}_{q} a_{r} a_{s} $$


- $p,q,r,s$ loops over the sets of spin orbitals: $ \{ \chi_{i} \}_{i=1,2,...,2K}$

In [ ]:
from openfermion.ops import FermionOperator

In [ ]:
# two_body_terms = two_body_terms.transpose(0,2,3,1) # turn into physist notation

In [ ]:
H_fermionic = FermionOperator((),  full_system_mol.energy_nuc())

# one body terms
for p in range(one_body_terms.shape[0]):
    for q in range(one_body_terms.shape[0]):
        
        H_fermionic += one_body_terms[p,q] * FermionOperator(((p, 1), (q, 0)))
        
        # two body terms
        for r in range(two_body_terms.shape[0]):
            for s in range(two_body_terms.shape[0]):
                
                ######## physist notation
                ## (requires:
                ##           two_body_terms = two_body_terms.transpose(0,2,3,1) before loop starts!
                ##)
#                 H_qubit += 0.5*two_body_terms[p,q,r,s] * FermionOperator(((p, 1), (q, 1), (r,0), (s, 0)))
                
                ######## chemist notation
                H_fermionic += 0.5*two_body_terms[p,q,r,s] * FermionOperator(((p, 1), (r, 1), (s,0), (q, 0)))
H_fermionic

In [ ]:
from openfermion.transforms import jordan_wigner

H_qubit_JW = jordan_wigner(H_fermionic)
H_qubit_JW

In [ ]:
# too expensive for desktop if molecule is H2O!
from openfermion.linalg import get_sparse_operator

if geometry != """
O
H 1 1.1
H 1 1.1 2 104
""":
    H_JW_mat = get_sparse_operator(H_qubit_JW)
    eigvals, eigvecs = np.linalg.eigh(H_JW_mat.todense())
    
    idx = eigvals.argsort()   
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:,idx]
    
    print(min(eigvals))

# Check Hartree Fock ground state

In [ ]:
from functools import reduce

In [ ]:
zero_state = np.array([[1],[0]])
one_state = np.array([[0],[1]])

HF_state = reduce(np.kron, [one_state, one_state, zero_state, zero_state]) # | 1 1 0 0 >
HF_state

In [ ]:
np.array_equal(np.eye(2**4)[int('1100',2), :].reshape(2**4,1), HF_state)

In [ ]:
HF_state.conj().T @ H_JW_mat.todense() @ HF_state

In [ ]:
FCI_ground_state = eigvecs[:,0]

row_inds, _ = np.where(FCI_ground_state!=0)

In [ ]:
for state_ind in row_inds:
    print(f' {eigvecs[state_ind, 0]} * |{np.binary_repr(state_ind,width=n_qubits)}>')

In [ ]:
zero_state = np.array([[1],[0]])
one_state = np.array([[0],[1]])

FCI_state = (0.11254388689315997*reduce(np.kron, [zero_state, zero_state, one_state, one_state]) + # | 0 0 1 1 >
             -0.9936467548998383*reduce(np.kron, [one_state, one_state, zero_state, zero_state])   # | 1 1 0 0 >
            )

FCI_state.conj().T @ H_JW_mat.todense() @ FCI_state

# PySCF FCI calc

In [ ]:
from pyscf import fci

HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose=1
HF_scf.max_memory= 8_000
HF_scf.conv_tol = 1e-6
HF_scf.kernel()

my_fci = fci.FCI(HF_scf).run()
print('E(UHF-FCI) = %.12f' % my_fci.e_tot)

In [ ]:
np.abs(FCI_state.conj().T @ H_JW_mat.todense() @ FCI_state - my_fci.e_tot)

In [ ]:
my_fci.norb



In [ ]:
# dm1 = my_fci.make_rdm1(my_fci.ci, my_fci.norb, my_fci.nelec)

In [ ]:
one_body_terms

In [ ]:
# dm2 = my_fci.make_rdm2(my_fci.ci, my_fci.c, my_fci.nelec)

In [ ]:
# help(my_fci.make_rdm2)

## PySCF CC

https://psicode.org/psi4manual/master/cc.html

In [ ]:
from pyscf import cc

In [ ]:
embedded_cc_obj = cc.UCCSD(HF_scf) # note UCCSD is unrestriced and CCSD is RESTRICTED!

e_cc, t1, t2 = embedded_cc_obj.kernel()


In [ ]:
HF_state = reduce(np.kron, [one_state, one_state, zero_state, zero_state]) # | 1 1 0 0 >
HF_state

$$T_{1} = \sum_{i}^{occ} \sum_{a}^{virt} t_{i}^{a}|\Psi_{i}^{a} >$$

$$T_{2} = \sum_{i}^{occ} \sum_{j>i}^{occ} \sum_{a}^{virt} \sum_{b>a}^{virt} t_{ii}^{ab}|\Psi_{ii}^{ab} >$$

In [ ]:
from copy import deepcopy

In [ ]:
t1[0]

In [ ]:
embedded_cc_obj.nmo

In [ ]:
int('0011',2)

In [ ]:
t_13 = t1[0,0]
print(f' {t_13} * |{np.binary_repr(6,width=n_qubits)}>')

In [ ]:
t_01_23 = t2[0,0,0,0]
print(f' {t_01_23} * |{np.binary_repr(3,width=n_qubits)}>')

In [ ]:
HF_string = np.binary_repr(12,width=n_qubits)
HF_amplitude = np.sqrt(1 - (t1[0,0]**2 + t2[0,0,0,0]**2))
HF_amplitude

In [ ]:
print('CCSD ground state')
print(f' {HF_amplitude} * |{HF_string}> + {t_13} * |{np.binary_repr(6,width=n_qubits)}> + {t_01_23} * |{np.binary_repr(3,width=n_qubits)}> ')

In [ ]:
t1a

In [ ]:
sum(embedded_cc_obj.nocc)

In [ ]:
embedded_cc_obj.nocc

In [ ]:
sum(embedded_cc_obj.nocc)

In [ ]:
t2aa.shape

In [ ]:
ia_OCC_spin_up_inds = np.arange(0, 2*embedded_cc_obj.nocc[0], 2)
ia_VIR_spin_up_vir_inds = np.arange(2*embedded_cc_obj.nocc[0],2*(sum(embedded_cc_obj.nmo) - sum(embedded_cc_obj.nocc)) , 2)

ib_OCC_spin_down_inds = np.arange(1, 2*embedded_cc_obj.nocc[1], 2)
ib_VIR_spin_down_inds = np.arange(2*embedded_cc_obj.nocc[1]+1, 2*(sum(embedded_cc_obj.nmo) - sum(embedded_cc_obj.nocc)), 2)

In [ ]:
ia_VIR_spin_up_vir_inds

In [ ]:
# HF_string = np.binary_repr(12,width=n_qubits)

### different spins!
t1a, t1b = t1
t2aa, t2ab, t2bb = t2

## single spin up


## single spin up
for i, t_ia in enumerate(t1a):
    i_ind = ia_OCC_spin_up_inds[i]
    a_ind = ia_VIR_spin_up_vir_inds[i]
    
    state_string = list(HF_string)
    state_string[i_ind]='0'
    state_string[a_ind]='1'
    state_string = ''.join(state_string)
    print(f' {t_ia} * |{state_string}>')

        
## single spin down
for i, t_ib in enumerate(t1b):
    i_ind = i_spin_down_occ_inds[i]
    b_ind = i_spin_up_vir_inds[i]
    
    state_string = list(HF_string)
    state_string[i_ind]='0'
    state_string[b_ind]='1'
    state_string = ''.join(state_string)
    print(f' {t_ib} * |{state_string}>')

    
# spin up spin down
for i, t_ijab in enumerate(t2ab):
    
    i_ind = ia_OCC_spin_up_inds[i]
    j_ind = ib_OCC_spin_down_inds[i]
    
    a_ind = ia_VIR_spin_up_vir_inds[i]
    b_ind = ib_VIR_spin_down_inds[i]
    
    state_string = list(HF_string)
    state_string[i_ind]='0'
    state_string[j_ind]='0'

    state_string[a_ind]='1'
    state_string[b_ind]='1'
    state_string = ''.join(state_string)
    print(f' {t_ijab} * |{state_string}>')
        


# # spin up spin up
# for i, t_ijaa in enumerate(t2aa):
#     if not bool(i%2):
#         continue
#     for j in range(i, len(t2aa)):
#         for a in range(2*embedded_cc_obj.nocc[0], (2*embedded_cc_obj.nmo[0]-embedded_cc_obj.nocc[0])):
#             if not bool(a%2):
#                 continue
#             for b in range(a, (2*embedded_cc_obj.nmo[0]-embedded_cc_obj.nocc[0])):
                
#                 state_string = list(HF_string)
#                 state_string[i]='0'
#                 state_string[j]='0'
                
#                 state_string[a]='1'
#                 state_string[b]='1'
#                 state_string = ''.join(state_string)
#                 print(f' {t_ijaa} * |{state_string}>')
                
                
# # spin down spin down
# for i, t_ijab in enumerate(t2bb):
#     if bool((i+1)%2):
#         continue
#     for j in range(i, len(t2bb)):
#         for a in range(2*embedded_cc_obj.nocc[1], (2*embedded_cc_obj.nmo[1]-embedded_cc_obj.nocc[1])):
#             if bool((a+1)%2):
#                 continue
#             for b in range(a, (2*embedded_cc_obj.nmo[1]-embedded_cc_obj.nocc[1])):
                
#                 state_string = list(HF_string)
#                 state_string[i+1]='0'
#                 state_string[j+1]='0'
                
#                 state_string[a+1]='1'
#                 state_string[b+1]='1'
#                 state_string = ''.join(state_string)
#                 print(f' {t_ijaa} * |{state_string}>')
                
# # spin up spin down
# for i, t_ijab in enumerate(t2ab):
#     if not bool(i%2):
#         continue
#     for j in range(i, len(t2bb)):
#         if not bool((j+1)%2):
#             continue
#         for a in range(2*embedded_cc_obj.nocc[1], (2*embedded_cc_obj.nmo[1]-embedded_cc_obj.nocc[1])):
#             if not bool(a%2):
#                 continue
#             for b in range(a, (2*embedded_cc_obj.nmo[1]-embedded_cc_obj.nocc[1])):
#                 if bool((b+1)%2):
#                     continue
#                 state_string = list(HF_string)
#                 state_string[i]='0'
#                 state_string[j]='0'
                
#                 state_string[a]='1'
#                 state_string[b]='1'
#                 state_string = ''.join(state_string)
#                 print(f' {t_ijaa} * |{state_string}>')

In [ ]:
bool(4%3)

In [ ]:
t2[2][0][0][0]

In [ ]:
t1